<a href="https://colab.research.google.com/github/substobeme/ML-Code-Analyzer/blob/main/ML_Code_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import subprocess
import sys
print("Installing dependencies")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'transformers', 'torch', 'accelerate'], check=True)

from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

Installing dependencies
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.7 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.
2026-01-07 03:51:03.623755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767757863.813495      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767757863.868593      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to regi

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
print("Loading models\n")

models = {}

models['TinyLlama'] = pipeline(
    'text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device_map='auto'
)

models['Phi-2'] = pipeline(
    'text-generation',
    model='microsoft/phi-2',
    device_map='auto',
    trust_remote_code=True,
    torch_dtype='auto'
)

print("Loading Qwen (1.8B)")
models['Qwen'] = pipeline(
    'text-generation',
    model='Qwen/Qwen2.5-1.5B-Instruct',
    device_map='auto',
    torch_dtype='auto'
)

print("All models loaded!\n")

Loading models



Device set to use cuda:0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Falling back to torch.float32 because loading with the original dtype failed on the target device.
Device set to use cpu


Loading Qwen (1.8B)


Device set to use cpu


All models loaded!



In [ ]:
print("Loading model")

generator = pipeline(
    'text-generation',
    model='microsoft/phi-2',
    device_map='auto',
    trust_remote_code=True,
    torch_dtype='auto'
)

print("model ready!\n")

Loading model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Falling back to torch.float32 because loading with the original dtype failed on the target device.
Device set to use cpu


model ready!



In [ ]:
def analyze_with_model(code, model_name, generator, num_chains=5):
    lines = code.strip().split('\n')
    numbered_code = '\n'.join(f"{i+1}: {line}" for i, line in enumerate(lines))

    cot_prompt = f"""Analyze this ML code line by line. Reference specific line numbers.

    CODE:
    {numbered_code}

    Analyze step by step:

    Step 1: Data Loading (lines 1-3)
    What issues in these specific lines?

    Step 2: Preprocessing (line 4)
    What's missing from this line?

    Step 3: Feature Selection (lines 5-6)
    Issues with these exact lines?

    Step 4: Train-Test Split (line 7)
    What parameter is missing?

    Step 5: Model (lines 8-9)
    Problems with this code?

    Step 6: Specific Fixes
    Give exact code changes.

    Be concrete. Quote line numbers."""

    chains = []
    print(f"  Generating {num_chains} chains for {model_name}")

    for i in range(num_chains):
        temp = 0.6 + (i * 0.05)
        print(f"Chain {i+1}/{num_chains} (temp={temp:.2f})...")

        result = generator(
            cot_prompt,
            max_new_tokens=800,
            temperature=temp,
            do_sample=True,
            pad_token_id=generator.tokenizer.eos_token_id,
            top_p=0.9
        )

        reasoning = result[0]['generated_text']

        words = set([
            w.lower() for w in reasoning.split()
            if len(w) > 3 and w.isalpha()
        ])

        chains.append({
            'id': i + 1,
            'reasoning': reasoning,
            'words': words
        })


    for chain in chains:
        agreement_score = 0
        for other in chains:
            if chain['id'] != other['id']:
                intersection = len(chain['words'] & other['words'])
                union = len(chain['words'] | other['words'])
                similarity = intersection / union if union > 0 else 0
                agreement_score += similarity
        chain['agreement_score'] = agreement_score / (num_chains - 1)


    best_chain = max(chains, key=lambda x: x['agreement_score'])

    return {
        'model_name': model_name,
        'all_chains': chains,
        'best_chain': best_chain,
        'avg_agreement': sum(c['agreement_score'] for c in chains) / len(chains)
    }

In [ ]:
def analyze_ml_code(code, num_chains=20):


    cot_prompt = f"""<|system|>
    You are an expert ML engineer. Use step-by-step reasoning to analyze code.</s>
    <|user|>
    Analyze this ML code using chain of thought. Think through each step explicitly.
    ```python
    {code}
    ```

    Let's think step by step:

    Step 1: Understanding the Goal
    What is this code trying to accomplish?

    Step 2: Data Analysis
    Examine the data handling and loading.

    Step 3: Preprocessing Quality
    Evaluate the preprocessing steps.

    Step 4: Model Selection
    Is the model choice appropriate?

    Step 5: Training & Evaluation
    How is training and evaluation done?

    Step 6: Identify Issues
    What problems or risks exist?

    Step 7: Recommendations
    What improvements should be made?

    Work through each step carefully, explaining your reasoning.</s>
    <|assistant|>
    Let me think through this step by step:

    Step 1: Understanding the Goal
    """

    chains = []

    print(f"GENERATING {num_chains} CHAINS OF THOUGHT\n")


    for i in range(num_chains):
        temp = 0.6 + (i * 0.03)
        print(f"Chain {i+1}/{num_chains} (temp={temp:.2f})...")

        result = generator(
            cot_prompt,
            max_new_tokens=1000,
            temperature=temp,
            do_sample=True,
            pad_token_id=generator.tokenizer.eos_token_id,
            top_p=0.9
        )


        cot_text = result[0]['generated_text']
        if '<|assistant|>' in cot_text:
            reasoning = cot_text.split('<|assistant|>')[-1].strip()
        else:
            reasoning = cot_text


        words = set([
            w.lower() for w in reasoning.split()
            if len(w) > 3 and w.isalpha()
        ])

        chains.append({
            'id': i + 1,
            'reasoning': reasoning,
            'words': words,
            'length': len(reasoning.split())
        })

    print(f"Generated {num_chains} chains\n")


    print("CALCULATING SELF-CONSISTENCY SCORES\n")

    for chain in chains:
        agreement_score = 0

        for other_chain in chains:
            if chain['id'] != other_chain['id']:
                intersection = len(chain['words'] & other_chain['words'])
                union = len(chain['words'] | other_chain['words'])
                similarity = intersection / union if union > 0 else 0
                agreement_score += similarity

        chain['agreement_score'] = agreement_score / (num_chains - 1)

        print(f"Chain {chain['id']}: Agreement score = {chain['agreement_score']:.3f}")

    chains_sorted = sorted(chains, key=lambda x: x['agreement_score'], reverse=True)

    print("\n" + "=" * 80)
    print("TOP 5 CHAINS:\n")

    for i, chain in enumerate(chains_sorted[:5]):
        print(f"{i+1}. Chain {chain['id']} - Agreement: {chain['agreement_score']:.3f}")

    print("\n" + "=" * 80)

    best_chain = chains_sorted[0]

    print(f"BEST CHAIN: Chain {best_chain['id']}")
    print(f"   Agreement Score: {best_chain['agreement_score']:.3f}")
    print(f"   This chain has the most overlap with other chains (self-consistency)")
    print("=" * 80)
    print(best_chain['reasoning'])
    print("\n" + "=" * 80)

    return {
        'all_chains': chains_sorted,
        'best_chain': best_chain,
        'best_reasoning': best_chain['reasoning']
    }

In [ ]:
def compare_models(code, num_chains=5):

    print("MULTI-MODEL CHAIN OF THOUGHT ANALYSIS\n")
    print("=" * 80)

    results = {}

    for model_name, generator in models.items():
        print(f"\nAnalyzing with {model_name}")
        results[model_name] = analyze_with_model(code, model_name, generator, num_chains)
        print(f" Best agreement: {results[model_name]['best_chain']['agreement_score']:.3f}")
        print(f" Avg agreement: {results[model_name]['avg_agreement']:.3f}")

    print("\n\n" + "=" * 80)
    print("MODEL COMPARISON")
    print("=" * 80)

    for model_name, result in results.items():
        print(f"\n{model_name}:")
        print(f"  Agreement Score: {result['best_chain']['agreement_score']:.3f}")
        print(f"  Average Agreement: {result['avg_agreement']:.3f}")

    print("\n\n" + "=" * 80)
    print("🏆 BEST ANALYSIS FROM EACH MODEL")
    print("=" * 80)

    for model_name, result in results.items():
        print(f"\n{'='*80}")
        print(f"MODEL: {model_name}")
        print(f"Agreement Score: {result['best_chain']['agreement_score']:.3f}")
        print(f"{'='*80}")
        print(result['best_chain']['reasoning'])
        print("...\n")

    best_model = max(results.items(), key=lambda x: x[1]['best_chain']['agreement_score'])

    print("\n" + "=" * 80)
    print(f"OVERALL WINNER: {best_model[0]}")
    print(f"   Highest agreement score: {best_model[1]['best_chain']['agreement_score']:.3f}")
    print("=" * 80)
    print("\nFULL ANALYSIS:")
    print(best_model[1]['best_chain']['reasoning'])

    return results

In [ ]:
example_code = """
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('indianmedico.csv')
df['Age'].fillna(df['Age'].mean(), inplace=True)
df = pd.get_dummies(df, columns=['Sex'])

X = df.drop(['Name'], axis=1)
y = df['Disease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
"""

In [ ]:
results = compare_models(example_code, num_chains=2)

MULTI-MODEL CHAIN OF THOUGHT ANALYSIS


Analyzing with TinyLlama
  Generating 2 chains for TinyLlama
Chain 1/2 (temp=0.60)
Chain 2/2 (temp=0.65)
 Best agreement: 1.000
 Avg agreement: 1.000

Analyzing with Phi-2
  Generating 2 chains for Phi-2
Chain 1/2 (temp=0.60)
Chain 2/2 (temp=0.65)
 Best agreement: 0.773
 Avg agreement: 0.773

Analyzing with Qwen
  Generating 2 chains for Qwen
Chain 1/2 (temp=0.60)
Chain 2/2 (temp=0.65)
 Best agreement: 0.311
 Avg agreement: 0.311


MODEL COMPARISON

TinyLlama:
  Agreement Score: 1.000
  Average Agreement: 1.000

Phi-2:
  Agreement Score: 0.773
  Average Agreement: 0.773

Qwen:
  Agreement Score: 0.311
  Average Agreement: 0.311


🏆 BEST ANALYSIS FROM EACH MODEL

MODEL: TinyLlama
Agreement Score: 1.000
Analyze this ML code line by line. Reference specific line numbers.

    CODE:
    1: import pandas as pd
2: from sklearn.model_selection import train_test_split
3: from sklearn.ensemble import RandomForestClassifier
4: from sklearn.metrics import a

In [ ]:
analyze_ml_code(example_code)

GENERATING 20 CHAINS OF THOUGHT

Chain 1/20 (temp=0.60)...
Chain 2/20 (temp=0.63)...
Chain 3/20 (temp=0.66)...
Chain 4/20 (temp=0.69)...
Chain 5/20 (temp=0.72)...
Chain 6/20 (temp=0.75)...
Chain 7/20 (temp=0.78)...
Chain 8/20 (temp=0.81)...
Chain 9/20 (temp=0.84)...
Chain 10/20 (temp=0.87)...
Chain 11/20 (temp=0.90)...
Chain 12/20 (temp=0.93)...
Chain 13/20 (temp=0.96)...
Chain 14/20 (temp=0.99)...
Chain 15/20 (temp=1.02)...
Chain 16/20 (temp=1.05)...
Chain 17/20 (temp=1.08)...
Chain 18/20 (temp=1.11)...
Chain 19/20 (temp=1.14)...
Chain 20/20 (temp=1.17)...
Generated 20 chains

CALCULATING SELF-CONSISTENCY SCORES

Chain 1: Agreement score = 0.233
Chain 2: Agreement score = 0.329
Chain 3: Agreement score = 0.011
Chain 4: Agreement score = 0.328
Chain 5: Agreement score = 0.309
Chain 6: Agreement score = 0.301
Chain 7: Agreement score = 0.319
Chain 8: Agreement score = 0.233
Chain 9: Agreement score = 0.307
Chain 10: Agreement score = 0.259
Chain 11: Agreement score = 0.304
Chain 12: Agr

{'all_chains': [{'id': 2,
   'reasoning': "Let me think through this step by step:\n    \n    Step 1: Understanding the Goal\n    The goal of this code is to predict the disease based on the given dataset.\n    \n    Step 2: Data Analysis  \n    The code loads the dataset using pandas and fills the missing values in the 'Age' column with the mean value. It also creates dummy variables for the 'Sex' column.\n    \n    Step 3: Preprocessing Quality\n    The code has a good approach to handle missing values and categorical data.\n    \n    Step 4: Model Selection\n    The RandomForestClassifier is a good choice for this problem as it is a classification model and can handle both categorical and numerical data.\n    \n    Step 5: Training & Evaluation\n    The code splits the dataset into training and testing sets and trains the model on the training set. It then evaluates the model's performance on the testing set using the accuracy score.\n    \n    Step 6: Identify Issues\n    There are